## Instructions

In order to run this program, please download the files provided at http://www-imai.is.s.u-tokyo.ac.jp/~hmiyata/oriented_matroids/ and unzip the files in the folder.

## Initial functions

In [1]:
"""
Get the list of facets from a file:
"""
def getFacets(fname):
    if isinstance(fname,str):
        try:
            return getFacets(open(fname,"r"))
        except:
            return []
    
    return [[Set(eval(f)) for f in l.split()] for l in fname]

In [2]:
def intersect(*args):
    if(len(args)) == 1:
        return args[0]
    return args[0].intersection(intersect(*args[1:]))

def union(*args):
    if(len(args)) == 1:
        return args[0]
    return args[0].union(intersect(*args[1:]))

In [3]:
"""
Given a facet-vertex incidence, give the edges.

This works as follows:
- take all possible pairs of vertices (C(n,2));
- For each pair, take all the facets containing both vertices
- The  intersection of these facets is precisely these two vertices if and only if it's an edge.
"""
def findEdges(facets):
    vertices = union(*facets)
    #take all pairs of vertices:
    V = [(v,w) for v in vertices for w in vertices if v<w]
    L = []
    U = Set(vertices)
    for (v,w) in V:
        F = [f for  f in facets if v in f and w in f]
        E = intersect(*F)
        if E.cardinality() == 2:
            L.append(E)
    return [tuple(e) for e in L]

## Hajòs' conjecture

In this section we verify that the graphs of all "small" polytopes satisfy Hajòs' conjecture.

In [1]:
def findLargestCompleteTopologicalMinor(H,n):
    k = n
    while H.topological_minor(graphs.CompleteGraph(k)) :
        k = k+1
    return k

def findLargestCompleteMinor(H,n):
    k = n
    while not (H.minor(graphs.CompleteGraph(k))) :
        k = k-1
    return k

In [10]:
"""
Check Hajos' conjecture on graphs of polytopes. More specifically:
- Compute  the chromatic number t of the graph;
- Check that the graph has K_t as a minor.
"""
def checkHajos(fname):
    fpol = open(fname,"r")
    lines = fpol.readlines()
    fpol.close()
    polys = []
    for l in lines:
       P = Polyhedron(vertices=eval(l[l.index("["):]),base_ring=QQ)
       G = P.graph()
       C = G.chromatic_number();
       #m = findLargestCompleteTopologicalMinor(G,6)
       m = G.minor(graphs.CompleteGraph(C))
       #t = findLargestCompleteMinor(G,10)
       polys.append((P,G,(C,m)))
    return polys

In [ ]:
any([d[2] == false for dat in hajos for d in dat])

In [4]:
"""
Check Hajos' conjecture on graphs of polytopes. More specifically:
- Compute  the chromatic number t of the graph;
- Check that the graph has K_t as a minor.
"""
def checkHajos2(fname,nvertices):
    polys = []
    for facets in getFacets(fname):
        if facets:
            edges = findEdges(facets)
            G = Graph(edges)
            C = G.chromatic_number();
            #m = findLargestCompleteTopologicalMinor(G,6)
            m = G.minor(graphs.CompleteGraph(C))
            #t = findLargestCompleteMinor(G,10)
            polys.append((facets,G,(C,m)))
    return polys

In [5]:
hajos = [checkHajos2("polytope_n%dd%d.txt"%(n,d),n) for n in range(6,10) for d in range(4,8)]

hajos2 = [h for h in hajos if h != []]

any([d[2] == false for dat in hajos2 for d in dat])

False

In [11]:
hajosSimplex = checkHajos("polytopalsimplicialspheres9_4.txt")

any([d[2] == false for d in hajosSimplex])

False

## Thomson's conjecture on edge colouring

We check Thomson's conjecture on "small" polytopes. Namely, check that it applies on the duals of simplicial polytopes.

In [13]:
"""
This function returns the facets of the dual polytope, given a list of facets for the primal.
"""
def dualFacets(f):
    nvertices = len(f)
    facets = [Set([i for (i,facet) in enumerate(f) if v in facet]) for v in union(*f)]
    return facets

In [19]:
"""
Check Thomson's conjecture on the dual of a d-polytope:
"""
def checkThompson(fname,d):
    polytopes = getFacets(fname)
    simplicialFacets = [facets for facets in polytopes if all(f.cardinality() == d for f in facets)]
    evensimplificialFacets = [facets for facets in simplicialFacets if len(facets)%2 == 0]
    edgeslists = [findEdges(facets) for facets in evensimplificialFacets]
    graphs = [Graph(edges) for edges in edgeslists]
    return [(d,graph_coloring.edge_coloring(G,value_only=true)) for G in graphs]

In [20]:
thompson = [checkThompson("polytope_n%dd%d.txt"%(n,d),d) for n in range(6,10) for d in range(4,8)]

In [25]:
any([x[0]<x[1] for x in sum(thompson,[])])

False